# Assignment 2
You will continue to work with the hue data files supplied for Assignment 1. We assume that the folder that you work in has the following structure.
<code>
assignment02.ipynb
hue_upload.csv
hue_upload2.csv
</code>

The first four columns represent the `row id`, `user id`, `event id`, and `value`. Any extra columns are irrelevant. For example, the first row of one file reads:

`"1";"10";"lamp change 29 mei 2015 19 08 33 984";"OFF"`

As you can see, the `event id` encompasses both a description of the event (`lamp_change`) and the date/time
(May 29, 7:08:33 pm). The following events are considered informative:

| String               | Description                                               |
-----------------------|------------------------------------------------------------
| `lamp_change`          | Light control via app                                     |
| `nudge_time`           | Automatic light dim time for people in experimental group |
| `bedtime_tonight`      | Intended bedtime (self-reported)                          |
| `risetime`             | Rise time (self-reported)                                 |
| `rise_reason`          | Reason for rising (self-reported)                         |
| `adherence importance` | Adherence (self-reported)                                 |
| `fitness`              | Fitness (self-reported)                                   |

All self-reported values are entered around noon. Records with other events may be ignored.

In [3]:
import datetime as datetime
import pandas as pd
import numpy as np
import pymongo
import re 

pd.options.display.max_rows = 20

## Exercise 1 (70 points)
The first part of this assignment is to write a Python function `read_csv_data` that reads the data into
a Pandas DataFrame. The index should be a (date, user) tuple, where date is stored in datetime.datetime format
(see the document with the "Tips"). The columns of your Pandas DataFrame should be `bedtime`, `intended_bedtime`,
`rise_time`, `rise_reason`, `fitness`, `adherence_importance` and `in_experimental_group`. Note: it is important to stick to this nomenclature. The way to do this is by going through the CSV data line by line, and parsing each line
individually, following these requirements:

<ul>
<li>
`bedtime` should be inferred from the `lamp_change` event. There are multiple reasonable ways to accomplish this. In this assigment, we define the  bedtime as the last OFF state of the `lamp_change` in the interval between 7 pm of the current day and 6 am of the next day. For example, from the row printed above you may
infer that the person did not sleep before 7:08:33 pm. As you go through the lines in the csv file, whenever
you discover new relevant information, you either update an existing record in the dataframe (if a record
for that day and user exists), or you create a new record (see the document with the "Tips").
<br><br>
For example, if you encounter a line where user 10 turns the light on at 9 pm and another line where he
turns it off at 10 pm (still on May 29), you update the record above to change the bedtime to 10 pm. If
someone falls asleep past midnight, the bedtime should be stored in the record corresponding to the day before. Again, dates and times should be stored as datetime.datetime.
<br><br></li>
<li>`intended_bedtime` should be filled in based on the `bedtime_tonight` event. Note that 1030 probably means 10:30 in the evening. Again, dates and times should be stored as datetime.datetime.
<br><br></li>
<li>
`rise_time`. The value for the column in your solution should be obtained from the `risetime` event in the CSV file.
<br><br></li>
<li>
`rise_reason`, `fitness` and `adherence_importance` values should be copied from the CSV file. Note that if
multiple distinct values are entered, the last should be assumed to be correct.
<br><br></li>
<li>
`in_experimental_group` should be boolean (True/False). The default value is False, but should be changed
to True if a `nudge_time` event is encountered. If a user is in the experimental group on one day, he is on
all days.
</li>
</ul>    

In [ ]:
def read_csv_data(filenames):
# YOUR CODE HERE

# YOUR CODE ENDS HERE

In [ ]:
df = read_csv_data(['hue_upload.csv', 'hue_upload2.csv'])
df

In [ ]:
# check the index
print(df.index)


In [ ]:
# check the bedtime column
display(df[['bedtime']])


In [ ]:
# check the intended_bedtime column
display(df[['intended_bedtime']])


In [ ]:
# check the rise_time
display(df[['rise_time']])


In [ ]:
# check the rise_reason, fitness, adherence_importance column
display(df[['rise_reason', 'fitness', 'adherence_importance']])


In [ ]:
# check the in_experimental_group column
display(df[['in_experimental_group']])


## Exercise 2 (10 + 20 points)
The second part of this assignment is to store the contents of the DataFrame into MongoDB, and to write a function that retrieves data from MongoDB and outputs it in a user-friendly format.

<ol>
<li>
The data should be stored in the collection "sleepdata" in the database "BigData". Make sure to use
the same column names as specified for the DataFrame, and to define the correct primary key. See
the document with the "Tips" for some comments about the primary key. Add the extra columns "date",
"user", "sleep duration" to facilitate sorting the data if necessary. Here, "sleep duration" is the difference between the risetime and the bedtime.
<br><br></li>
<li>The following is an example of how the output must be presented.
    
| date | user | bedtime | intended | risetime | reason | fitness | adh | in_exp | sleep_duration |
-------|------|---------|----------|----------|--------|---------|-----|--------|----------------|    
| 11-06-2015 | 2  | 00:51:28 | 22:30:00 | 07:00:00 | ja  | -    | 47.0 | no  | 22351 |
| 11-06-2015 | 20 | 00:28:10 | 23:00:00 | 07:10:00 | nee | 55.0 | 88.0 | yes | 33510 |
| 11-06-2015 | 34 | 19:54:10 | -        | -        | -   | -    | -    | yes | -     |

Here sleep duration is in number of seconds. Note that, in order to determine the sleep duration of day X, it
is necessary to know the risetime of day X, but the bedtime of day X - 1.
<br><br></li>
</ol>

In [69]:
from datetime import datetime, timedelta, date
import random

start = datetime.now()
end = start + timedelta(days=5)

def r():
    return start + (end - start) * random.random()

today = date.today()
today_with_time = datetime(
    year=today.year, 
    month=today.month,
    day=today.day,
)

tomorrow_with_time = datetime(
    year=today.year, 
    month=today.month,
    day=today.day + 1,
)

d = { 'bedtime': [r(), r()]
    , 'intended_bedtime': [r(), r()]
    , 'rise_time': [r(), r()]
    , 'rise_reason': ["a", "b"]
    , 'fitness': ["c", "d"]
    , 'adherence_importance': ["very important", "not important"] 
    , 'in_experimental_group': [True, False]
    , 'index': [(tomorrow_with_time, "ola"), (today_with_time, "ola")] 
    }

df = pd.DataFrame(data=d)
df = df.set_index('index')

def to_mongodb():
# YOUR CODE HERE
    
    # connect to MongoDB database
    client = pymongo.MongoClient("localhost", 27017)
    db = client.BigData
    sleepdata = db.sleepdata
    sleepdata.delete_many({})
    
    # set primary key
    df['_id'] = pd.Series(dict(zip(df.index, list(map(lambda x : {"date": x[0], "name": x[1]}, df.index)))))
    
    # add column date and user
    df['user'] = pd.Series(dict(zip(df.index, list(map(lambda x : x[1], df.index)))))
    df['date'] = pd.Series(dict(zip(df.index, list(map(lambda x : x[0], df.index)))))
    
    # add sleep_duration column
    bedtime = df['bedtime'].transpose().to_dict()
    risetime = df['rise_time'].transpose().to_dict()
    
    def sleep_time(index):
        rise = risetime.get(index)
        bet = bedtime.get((index[0] - timedelta(days=1), index[1]))
        if rise and bet:
            return (rise - bet).total_seconds()
    
    df['sleep_duration'] = pd.Series(dict(zip(df.index, list(map(sleep_time, df.index)))))
    df.fillna("-",inplace=True)
    
    sleepdata.insert_many(df.to_dict('records'))
    
to_mongodb()    
# YOUR CODE ENDS HERE

In [61]:
to_mongodb(df)


TypeError: to_mongodb() takes 0 positional arguments but 1 was given

In [86]:
def read_mongodb(filter,sort):
# YOUR CODE HERE

    # connect to MongoDB database
    connection = pymongo.MongoClient("localhost", 27017)
    db = connection.BigData
    sleepdata = db.sleepdata
    data = sleepdata.find(filter).sort(sort, pymongo.ASCENDING)
    
    for row in data:
        print(row)
    
    return data
# YOUR CODE ENDS HERE

In [87]:
query = read_mongodb({'sleep_duration': {'$gt': 40000}}, '_id')
print(query)


AttributeError: 'Cursor' object has no attribute 'forEach'

[]
